In [2]:
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LassoCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

warnings.filterwarnings("ignore")

In [3]:
ma = pd.read_csv("../16 Mycoplosma/ma.csv")
map = pd.read_csv("../16 Mycoplosma/map.csv")
marth = pd.read_csv("../16 Mycoplosma/marth.csv")
mcap = pd.read_csv("../16 Mycoplosma/mcap.csv")
mcon = pd.read_csv("../16 Mycoplosma/mcon.csv")
mcro = pd.read_csv("../16 Mycoplosma/mcro.csv")
mgal = pd.read_csv("../16 Mycoplosma/mgal.csv")
mhom = pd.read_csv("../16 Mycoplosma/mhom.csv")
mhyoJ = pd.read_csv("../16 Mycoplosma/mhyoJ.csv")
mhyo232 = pd.read_csv("../16 Mycoplosma/mhyo232.csv")
mhyo7448 = pd.read_csv("../16 Mycoplosma/mhyo7448.csv")
mm163K = pd.read_csv("../16 Mycoplosma/mm163K.csv")
mms = pd.read_csv("../16 Mycoplosma/mms.csv")
mpHF = pd.read_csv("../16 Mycoplosma/mpHF.csv")
mpM = pd.read_csv("../16 Mycoplosma/mpM.csv")
ms53 = pd.read_csv("../16 Mycoplosma/ms53.csv")

In [4]:
def generate_train_test_dataset(index):
    dataset_names = [ma, map, marth, mcap, mcon, mcro, mgal, mhom, mhyoJ, mhyo232, mhyo7448, mm163K, mms, mpHF, mpM, ms53]
    merged_train = dataset_names[:index]+dataset_names[index+1:]
    train_data = pd.concat(merged_train)
    test_data = dataset_names[index]
    return train_data,test_data

In [5]:
def handle_dataset(dataset):
    columns_to_drop = ['identity','name', 'Amino acid', 'protein_sequence', 'essentiality']
    Xs = dataset.drop(columns = columns_to_drop)
    y = dataset['essentiality']
    return Xs,y

In [8]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [9]:
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 700, 1),
    'max_depth': hp.quniform('max_depth', 3, 20, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),
}

In [10]:

from sklearn.metrics import accuracy_score
def rf_ac_cv(params):
    params = {
    'n_estimators': int(params['n_estimators']), 
    'max_depth': int(params['max_depth']), 
    'min_samples_split': params['min_samples_split'],
    'min_samples_leaf': params['min_samples_leaf']
}
    scorer = make_scorer(accuracy_score)
    model = RandomForestClassifier(random_state=42, **params)
    score = -cross_val_score(model, X_resampled[selected_features], y_resampled, cv=5,scoring=scorer, ).mean()
    return score

In [11]:
from sklearn.ensemble import RandomForestClassifier

scores = []
score = 0
models = []
features = []
for j in range(0,3):
    for i in range(0,15):
        train_data, test_data = generate_train_test_dataset(i)
        Xs_train,y_train = handle_dataset(train_data)
        Xs_test,y_test = handle_dataset(test_data)
        rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
        X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
        lassoCV.fit(X_resampled,y_resampled)
        selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
        trials = Trials()
        best=fmin(fn=rf_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=15, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
        )
        model = RandomForestClassifier(random_state=42, n_estimators=int(best['n_estimators']),max_depth=int(best['max_depth']),min_samples_split=best['min_samples_split'],min_samples_leaf=best['min_samples_leaf'])
        models.append(model)
        model.fit(X_resampled[selected_features],y_resampled)
        tpe_test_score=accuracy_score(y_test, model.predict(Xs_test[selected_features]))
        scores.append(tpe_test_score)
        print(tpe_test_score)
        if score < tpe_test_score:
            score = tpe_test_score
            best_model = model
            features = selected_features

100%|██████████| 15/15 [00:28<00:00,  1.88s/trial, best loss: -0.8085193889541715]
0.883289124668435
100%|██████████| 15/15 [00:28<00:00,  1.92s/trial, best loss: -0.8186813186813187]
0.8467908902691511
100%|██████████| 15/15 [00:29<00:00,  2.00s/trial, best loss: -0.8146774377062236]
0.6226415094339622
100%|██████████| 15/15 [00:28<00:00,  1.87s/trial, best loss: -0.7941028791197051]
0.9795918367346939
100%|██████████| 15/15 [00:27<00:00,  1.84s/trial, best loss: -0.8059053533436835]
0.901840490797546
100%|██████████| 15/15 [00:29<00:00,  1.94s/trial, best loss: -0.8051912568306012]
0.9322033898305084
100%|██████████| 15/15 [00:28<00:00,  1.91s/trial, best loss: -0.810705052878966] 
0.7627118644067796
100%|██████████| 15/15 [00:28<00:00,  1.93s/trial, best loss: -0.8102121787131275]
0.9516129032258065
100%|██████████| 15/15 [00:28<00:00,  1.91s/trial, best loss: -0.8019001024007283]
0.7976878612716763
100%|██████████| 15/15 [00:30<00:00,  2.00s/trial, best loss: -0.7931091008439075]
0

In [12]:
print(scores)

[0.883289124668435, 0.8467908902691511, 0.6226415094339622, 0.9795918367346939, 0.901840490797546, 0.9322033898305084, 0.7627118644067796, 0.9516129032258065, 0.7976878612716763, 0.8345428156748912, 0.8092485549132948, 0.9449035812672176, 0.9900249376558603, 0.97, 0.5728744939271255]


In [13]:
from joblib import dump

dump(best_model, '../../model/RF_model_lasso.joblib')

['../../model/RF_model_lasso.joblib']

In [14]:
print(features)

['GC_Content', 'CAI', 'A', 'N', 'D', 'H', 'M', 'nSE3', 'nGE3']
